# Modelo de Recomendación de películas

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel

In [2]:
df = pd.read_csv('../clean_data/movies_model.csv')
# x = 0
# for i in range(len(df)):
#     print(i)
#     if len(df.word_bag[i]) > x:
#         x = len(df.word_bag[i])
# print(x)

In [3]:
#stop_words attribute will skip all the helper words that occur english language eg: in, the , they etc.
# del(vector)
# del(cv_fvector)
# del(similarity_array)
cv_fvector = CountVectorizer(stop_words='english')
vector = cv_fvector.fit_transform(df['word_bag']).toarray()

In [4]:
similarity_array = cosine_similarity(vector,vector)
# similarity_array = linear_kernel(vector,vector)

MemoryError: Unable to allocate 27.9 GiB for an array with shape (24762, 151225) and data type float64

In [14]:
# Recomendaciones con cosine_similarity
recommended_five = sorted(list(enumerate(similarity_array[862])),reverse=True,key = lambda x: x[1])[1:6]
lista = []
for i in recommended_five:
    lista.append(i[0])
df.loc[lista]

,id_peli,title,original_language,popularity,vote_average,vote_count,release_year,overview,franquicia
11476,19244,Mr. Untouchable,en,1.102543,6.8,6.0,2007,The true-life story of a Harlem's notorious Ni...,NaN
27525,202198,It'S A Dog'S Life,en,0.058551,4.0,1.0,1955,"A bull terrier tells his life story, from the ...",NaN
34504,42291,The Buddha,en,0.386889,6.0,3.0,2010,This documentary for PBS by award-winning film...,NaN
9571,53367,Charlie: The Life And Art Of Charles Chaplin,en,10.351391,6.5,27.0,2003,"Brilliant, long in-the-works story of the life...",NaN
13223,33511,Nowhere Boy,en,5.087320,7.2,250.0,2009,The drama tells the story of Lennon's teenage ...,NaN


In [5]:
# Recomendaciones con linear_kernel
recommended_five = sorted(list(enumerate(similarity_array[862])),reverse=True,key = lambda x: x[1])[1:6]
lista = []
for i in recommended_five:
    lista.append(i[0])
df.loc[lista]

,id_peli,title,original_language,popularity,vote_average,vote_count,release_year,overview,franquicia
3506,48109,Our Song,en,0.546957,6.4,7.0,2000,Focusing on the bonding between three female (...,NaN
5457,123833,Possible Loves,pt,0.388631,6.4,6.0,2001,"Fifteen year ago, Carlos went to the cinema to...",NaN
23570,221488,8 Minutes Idle,en,4.398140,5.4,7.0,2012,Based upon Matt Thorne's award-winning novel a...,NaN
18698,107978,Cockpit,sv,0.618969,5.1,8.0,2012,After getting fired from his current job as a ...,NaN
34038,227970,Men Are From Mars... And That’S Where I’M Going!,pt,1.452670,6.7,30.0,2014,"Successful at work, Fernanda is an example of ...",NaN


In [5]:
df[df.id_peli == 862]

,id_peli,title,original_language,popularity,vote_average,vote_count,release_year,overview,franquicia
0,862,Toy Story,en,21.946943,7.7,5415.0,1995,"Led by Woody, Andy's toys live happily in his ...",Toy Story Collection
